In [ ]:
import pandas as pd
from datetime import datetime

# Note: To read .xlsx files, pandas requires the 'openpyxl' library.
# If you don't have it, run: pip install openpyxl

class StressScoreCalculator:
    """
    Calculates a 'Stress Score' for Uber trip requests based on historical data.

    The score now considers two primary factors:
    1.  Location Stress: Based on historical cancellation rates for a given area. (Weight: 60%)
    2.  Environmental Stress: Based on the weather conditions for the day. (Weight: 40%)
    """

    def __init__(self, cancellation_path, weather_path):
        """
        Initializes the calculator by loading data from separate Excel files.

        Args:
            cancellation_path (str): Path to the cancellation_rates .xlsx file.
            weather_path (str): Path to the weather_daily .xlsx file.
        """
        print("Initializing Simplified StressScoreCalculator...")
        try:
            print(f"Loading data from files...")
            self.cancellation_rates = pd.read_excel(cancellation_path).set_index('hexagon_id9')
            self.weather = pd.read_excel(weather_path).set_index(['date', 'city_id'])
            print("Data loaded successfully.")
        except FileNotFoundError as e:
            print(f"Error loading data: {e}. Please ensure the file paths are correct.")
            return
        except ValueError as e:
            print(f"Error reading an Excel file: {e}. Please ensure the files are correct.")
            return

        # Store min/max values from the cancellation data for normalization
        self.min_cancel_rate = self.cancellation_rates['cancellation_rate_pct'].min()
        self.max_cancel_rate = self.cancellation_rates['cancellation_rate_pct'].max()
        print("Initialization complete.")


    def _normalize(self, value, min_val, max_val, scale_min=1, scale_max=5):
        """Scales a value from its original range to a 1-5 scale."""
        if max_val - min_val == 0:
            return scale_min # Avoid division by zero
        return scale_min + (value - min_val) * (scale_max - scale_min) / (max_val - min_val)

    def _get_location_stress(self, pickup_hex_id9):
        """Calculates stress based on cancellation rates."""
        try:
            rate = self.cancellation_rates.loc[pickup_hex_id9, 'cancellation_rate_pct']
            return self._normalize(rate, self.min_cancel_rate, self.max_cancel_rate)
        except KeyError:
            # Return a neutral score if the hexagon isn't in our data
            return 2.5

    def _get_environmental_stress(self, date_str, city_id):
        """Calculates stress based on weather conditions."""
        weather_map = {'clear': 1, 'rain': 3, 'snow': 5}
        try:
            # The .weather DataFrame index is datetime, so we must convert the date string
            date_obj = pd.to_datetime(date_str)
            weather_condition = self.weather.loc[(date_obj, city_id), 'weather']
            return weather_map.get(weather_condition, 2) # Default to 2 if condition is unknown
        except KeyError:
            # Return a neutral score if we have no weather data
            return 2.5

    def calculate_stress_score(self, trip_request):
        """
        Calculates the final Stress Score and its components for a given trip request.

        Args:
            trip_request (dict): A dictionary containing trip details:
                                 {'pickup_hex_id9': str, 'city_id': int, 'start_time': str}

        Returns:
            dict: A dictionary containing the final score and the individual stress components.
        """
        # Extract trip info
        pickup_hex = trip_request['pickup_hex_id9']
        city = trip_request['city_id']
        start_time = datetime.strptime(trip_request['start_time'], '%Y-%m-%d %H:%M:%S')
        date_str = start_time.strftime('%Y-%m-%d')
        hour = start_time.hour

        # Get individual stress components
        location_stress = self._get_location_stress(pickup_hex)
        env_stress = self._get_environmental_stress(date_str, city)
        
        # Define new weights for each component
        weights = {
            'location': 0.6,
            'environment': 0.4
        }

        # Calculate weighted average
        final_score = (location_stress * weights['location'] +
                       env_stress * weights['environment'])

        print("\n--- Stress Score Calculation ---")
        #print(f"Request: Pickup in hex {pickup_hex} at {start_time}")
        #print(f"  - Location Stress (Cancellations): {location_stress:.2f} / 5")
        #print(f"  - Environmental Stress (Weather):  {env_stress:.2f} / 5")
        print(f"Stress Score: {final_score:.2f} / 5")
        
        return {
            "final_score": final_score,
            "location_stress": location_stress,
            "environmental_stress": env_stress
        }

    def explain_score(self, score_data):
        """
        Provides a driver-friendly explanation of the stress score.

        Args:
            score_data (dict): The dictionary returned by calculate_stress_score.
        """
        final_score = score_data['final_score']
        
       


if __name__ == '__main__':
    # --- FILE PATHS ---
    # Update these paths to where your .xlsx files are located
    CANCELLATION_FILE = "C:/Users/91984/Downloads/cancellation_rate.xlsx"
    WEATHER_FILE = "C:/Users/91984/Downloads/weather.xlsx"

    # --- INITIALIZATION ---
    # Initialization is now much faster
    calculator = StressScoreCalculator(CANCELLATION_FILE, WEATHER_FILE)

    # --- EXAMPLE USAGE ---
    # Simulate a few incoming trip requests to test the simplified calculator

    # Example 1: High cancellation area on a clear day
    stressful_location_trip = {
        'pickup_hex_id9': '8904406d7ed9715', # This hex has a high cancellation rate in the mock data
        'city_id': 1,
        'start_time': '2023-01-10 14:00:00' # Assuming a clear day
    }
    score_data_1 = calculator.calculate_stress_score(stressful_location_trip)
    calculator.explain_score(score_data_1)

    # Example 2: A trip during a snow day (high environmental stress)
    # Based on weather data, 2023-01-04 had snow in city 3
    snowy_trip = {
        'pickup_hex_id9': '89b5443252677be', # A hex with average cancellation
        'city_id': 3,
        'start_time': '2023-01-04 08:30:00'
    }
    score_data_2 = calculator.calculate_stress_score(snowy_trip)
    calculator.explain_score(score_data_2)

    # Example 3: A low-stress trip
    # Low cancellation area on a clear day
    calm_trip = {
        'pickup_hex_id9': '890212a91086353', # This hex has a very low cancellation rate
        'city_id': 1,
        'start_time': '2023-01-11 11:00:00'
    }
    score_data_3 = calculator.calculate_stress_score(calm_trip)
    calculator.explain_score(score_data_3)